In [1]:
import torch
from torch.nn import CrossEntropyLoss, Sigmoid, ReLU, Softsign
from TrainingEnvironments.FeedForwardTrainingEnvironment import FeedForwardTrainingEnvironment
from Datasets.GaussianSample import GaussianData

from Models.MLP import MLP
import torch.optim as optim 

import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Constant Model Parameters
midpoints = torch.tensor([

    [.1, .9],
    [.9, .1]

])


#Constant Environment Parameters
epochs = 250
batches = 200
batchsize = 10

criterion = CrossEntropyLoss()

# Experiment 1

In [3]:
# Variable Model Parameters
trials = 1
hidden_shapes_list = [

    [2,2],
    
    ]
activation_function = Softsign

data_columns = ["Trial", "Depth", "Width", "Epoch", "Accuracy", "Loss"]


for trial in range(trials):
    data = GaussianData(midpoints)

    for hidden_shapes in hidden_shapes_list:
        
        current_data_columns = data_columns + [f'Layer {i+1} Mean Activations' for i in range(len(hidden_shapes))] + [f'Layer {i+1} STD Activations' for i in range(len(hidden_shapes))] + [f'Layer {i+1} Mean Gradient' for i in range(len(hidden_shapes))] + [f'Layer {i+1} STD Gradient' for i in range(len(hidden_shapes))]
        print(current_data_columns)

        #Creation of Model and Optimizer
        model = MLP(midpoints.shape[1], hidden_shapes, midpoints.shape[0], activation_function)
        optimizer = optim.SGD(model.parameters(), lr=0.1)

        # Creation and Running of Training Environment
        print(hidden_shapes)
        Experiment1 = FeedForwardTrainingEnvironment(data, model, criterion, optimizer, epochs, batches, batchsize)
        loss_data, accuracy_data, mean_activation_data, std_activation_data, mean_gradient_data, std_gradient_data = Experiment1.trainModel()

        pd.DataFrame(torch.cat((
                        torch.stack((
                            trial*torch.ones(epochs),
                            len(hidden_shapes)*torch.ones(epochs),
                            hidden_shapes[0]*torch.ones(epochs),
                            torch.tensor(range(epochs)),
                            torch.tensor(accuracy_data),
                            torch.tensor(loss_data),
                        ), dim=1),                         
                        torch.tensor(mean_activation_data).T,
                        torch.tensor(std_activation_data).T,
                        torch.tensor(mean_gradient_data).T,
                        torch.tensor(std_gradient_data).T
                    ), dim=1),
                    columns=current_data_columns)#.to_csv(f"./CollectedData/Softsign/uniform_25_75/trial{str(trial+1)}_depth{len(hidden_shapes)}_width{hidden_shapes[0]}.csv")

['Trial', 'Depth', 'Width', 'Epoch', 'Accuracy', 'Loss', 'Layer 1 Mean Activations', 'Layer 2 Mean Activations', 'Layer 1 STD Activations', 'Layer 2 STD Activations', 'Layer 1 Mean Gradient', 'Layer 2 Mean Gradient', 'Layer 1 STD Gradient', 'Layer 2 STD Gradient']
[2, 2]
